The goal in this file is comparing the solution of regular Bounded-Makespan-Min-Cost with Fixed_x_Bounded_Makespan_Min_Cost and generate the figure automatically.

In [1]:
# importing packages
import numpy as np
import pandas as pd
from gurobipy import *


# from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS
# from bokeh.layouts import row, column, gridplot, widgetbox
# from bokeh.models.widgets import Panel, Tabs, Button

# from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import numpy as np
import pandas as pd

from gurobipy import *

# output_notebook()

In [2]:
ScnCount = [20, 50, 100, 200, 1000]
subScnCount = 20
maxScn = 1000

Number = range(13, 49)
SubNumber = [1]
directory2 = "./scenarioGameNormal/10_scenarioGame_" 

In [3]:
# reading data
def readData(readNumber, readSubNumber):  
        jobs = 30
        testNumber = readNumber # 1 to 48
        subTestNumber = readSubNumber # 1 to 10
        fileName = 'J' + str(jobs) + str(testNumber) + '_' + str(subTestNumber) + '.RCP'
        file = '../dataRCPSP/A' + str(jobs) + '/j' + str(jobs) + 'rcp/' + fileName
        my_cols = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
        df = pd.read_table(file, sep='\t',delim_whitespace=True, engine='python', names=my_cols)
        duration = df.iloc[2:, 0].values
        resourcesUsage = df.iloc[2:, 1:5].values
        successorsCount = df.iloc[2:, 5].values
        successors = df.iloc[2:, 6:9].values
        resourcesCount = int(df.iloc[0, 1])
        jobsCount = int(df.iloc[0,0])
        availableResources = df.iloc[1,0:resourcesCount].values
        return duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources

In [4]:
# model sets and parameters for FCT model
def FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scenarioCount, subScenarioCount):  
        activities = np.array(range(jobsCount))
        resources = np.array(range(resourcesCount))
        scenarios = np.array(range(scenarioCount))
        subScenarios = np.array(range(subScenarioCount))
        T = duration.sum()
        d = dict(zip(activities,resourcesUsage))
        M = 150
        N = availableResources.max()
        return activities, resources, d, T, M , N, scenarios, subScenarios

In [5]:
# # scenario generating Uniform
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     aMin = 1 - (12**0.5)/10 #20%
#     aMax = 1 + (12**0.5)/10
#     for s in scenarios:
#         for a in activities:
#             cr = np.random.rand()
#             durationScenarios[a,s] = duration[a] * (aMin + cr * (aMax-aMin))
#     p = dict(zip(activities,durationScenarios))
#     return p



# scenario generating Normal
def scenarioGen(scenarios, jobsCount, duration, activities):
    np.random.seed(0)
    durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    for s in scenarios:
        for a in activities:
            if duration[a] != 0:
                cr = np.random.normal(duration[a], duration[a]*0.1)
            else: cr = 0
            if cr >= 0:
                durationScenarios[a,s] = cr
            else: 
                durationScenarios[a,s] = duration[a]
    p = dict(zip(activities,durationScenarios))
    return p



# # scenario generating Left Skew 600
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(600, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Left Skew 8
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(8, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p

In [6]:
# define variables of FCT model
def addFCTVars(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f, o, e

In [7]:
# define variables of FCT model
def addFCTVarsMM(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f

In [8]:
# define variables of FCT model
def addFCTVarsXX(activities, resources, T, availableResources):  
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return s, f, o, e

In [9]:
# constraint 2.5 of FCT model
def addFCTConst1(activities, successors, x):    
        FCT.addConstrs(
            (x[activityI, activityJ] 
             ==
             1
             for activityI in activities if activityI != activities[-1]
             for activityJ in activities[successors[activityI][successors[activityI] > 0].astype(np.int) - 1]), 
            name = "NetworkRelations")

In [10]:
# constraint 2.6a of FCT model
def addFCTConst2a(activities, x, s, p, M, scenarios):  
        FCT.addConstrs(
            (s[activityJ, scenario] -  s[activityI, scenario] 
             >=
             p[activityI][scenario] - M * (1 - x[activityI, activityJ])
             for scenario in scenarios
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [11]:
# constraint 2.6a of FCT model
def addFCTConst2aMM(activities, x, s, ppp, M):  
        FCT.addConstrs(
            (s[activityJ] -  s[activityI] 
             >=
             ppp[activityI] - M * (1 - x[activityI, activityJ])
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [12]:
# constraint 2.6b of FCT model
def addFCTConst2b(activities, s, o, scenarios):  
        FCT.addConstrs(
            (s[activity, scenario] 
             >=
             o[activity]
             for scenario in scenarios
             for activity in activities), 
            name = "NetworkStartTimeAndOrderRelations")

In [13]:
# constraint 2.7 of FCT model
def addFCTConst3(activities, x, f, resources, N):  
        FCT.addConstrs(
            (f[activityI, activityJ, resource] -  N * x[activityI, activityJ] 
             <=
             0
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]
             for resource in resources), 
            name = "NetworkFlowRelations")

In [14]:
# constraint 2.8 of FCT model
def addFCTConst4(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             d[activityI][resource]
             for activityI in activities if activityI != activities[0] 
             for resource in resources), 
            name = "OutgoingFlows")

In [15]:
# constraint 2.9 of FCT model
def addFCTConst5(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             d[activityJ][resource]
             for activityJ in activities if activityJ != activities[-1] 
             for resource in resources), 
            name = "IngoingFlows")

In [16]:
# constraint 2.10a of FCT model
def addFCTConst6(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activities[0], activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "FirstFlow")

In [17]:
# constraint 2.10b of FCT model
def addFCTConst7(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activityI, activities[-1], resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "LastFlow")

In [18]:
# constraint for bounding makespan
def addFCTConst9(activities, scenarios, scenarioCount, s, makespan):  
        FCT.addConstrs(
            (quicksum(s[activities[-1], scenario] for scenario in scenarios)
             <=
             makespan * scenarioCount
             for alaki in range(0,1)
             ), 
            name = "Makespandddd")

In [19]:
# Objective of FCT model -- min cost
def addFCTObj(activities, s, scenarios, scenarioCount, o): 
        obj = (quicksum(s[activity, scenario] - o[activity] 
                              for activity in activities
                              for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [20]:
# Objective of FCT model -- min makespan det
def addFCTObjMM(activities, s):
  
        obj = (s[activities[-1]])
        FCT.setObjective(obj, GRB.MINIMIZE)

In [21]:
# Objective of FCT model -- min expected makespan
def addFCTObjdW(activities, s, scenarios, scenarioCount):
  
        obj = (quicksum(s[activities[-1], scenario] for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [22]:
for i in Number:
    for j in SubNumber: 
        
        
        try:


            #MM##########################################################################################################
            ################################# MIN makespan for DETERMINISTIC
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN makespan for DETERMINISTIC    step=1  :   " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ############################################################################################################1

            makespanDet = 0
            makespanExp = 0

            duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources = readData(i, j)
            activities, resources, d, T, M , N, scenarios, subScenarios = \
                FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, maxScn, subScnCount)
            pAll = scenarioGen(scenarios, jobsCount, duration, activities)
            pMM = list(range(jobsCount))
            for act in range(jobsCount):
                pMM[act] = sum(pAll[act]) / maxScn
            FCT = Model('FCT: MIN makespan for DETERMINISTIC')
            FCT.setParam('TimeLimit', 60)
            x, s, f = addFCTVarsMM(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2aMM(activities, x, s, pMM, M)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjMM(activities, s)
            FCT.optimize()        

    #.............................................. Save MM-X

            directory = directory2 + str(i) + "_" + str(j) + "/"
            !mkdir $directory
            if FCT.Status == 2:

                # Open a file X
                xoName = directory+ "MM-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|MM|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()       


            #dW############################################################################################################
            ##################### MIN E(makespan) for SCENARIO base | XdW bede
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ##############################################################################################################2

            subP = dict(zip(activities,np.array(list(pAll.values()))[:,:subScnCount]))
            FCT = Model('FCT: MIN makespan for SCENARIO base')
            FCT.setParam('TimeLimit', 600)
            x, s, f, o, e = addFCTVars(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, subP, M, subScenarios)
            addFCTConst2b(activities, s, o, subScenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjdW(activities, s, subScenarios, subScnCount)
            FCT.optimize()        

    #.............................................. Save dW-X

            if FCT.Status == 2:
                # Open a file X
                xoName = directory + "dW-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            # Calculations
            if FCT.MIPGap <= 10000:
                aaa = 0 # Expected Makespan
                for vvv in range(subScnCount):
                    aaa += s[jobsCount - 1, vvv].X
            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|dW|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(aaa / subScnCount,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else: 
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()                 





            for scnCount in ScnCount:


                #SdW###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of dW shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  " + str(i) + "_" + str(j) + " || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################3

                fileName =  "dW-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)


                p = dict(zip(activities,np.array(list(pAll.values()))[:,:scnCount]))


                FCT = Model('for fixed X: MIN scenario used makespan | use x of dW')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName = "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SdW|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanExp = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



                #SMM###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of MM shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  " + str(i) + "_" + str(j) + "  || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################4

                fileName =   "MM-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)

                FCT = Model('for fixed X: MIN scenario used makespan | use x of MM')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SMM|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanDet = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



        except Exception as err:
            print("Erro {}".format(err))

iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_13_1
    MIN makespan for DETERMINISTIC    step=1  :   13_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5109 rows, 5152 columns and 18495 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 1309 rows and 1322 columns
Presolve time: 0.10s
Presolved: 3800 rows, 3830 columns, 14003 nonzeros
Variable types: 3131 continuous, 699 integer (699 binary)

Root relaxation: objective 3.394632e+01, 2898 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.94632    0   25          -   33.94632   

 24979 15266 infeasible  297        90.00271   34.20526  62.0%  78.7  241s
 27775 17626     cutoff  181        90.00271   34.20526  62.0%  78.4  246s
 28120 17904   87.81361  191   17   90.00271   34.20526  62.0%  79.9  250s
 29923 19473   34.99583  103   34   90.00271   34.20526  62.0%  80.2  256s
 30699 20107   35.00139  153   32   90.00271   34.20526  62.0%  80.4  260s
 32717 21818   63.60075  120   28   90.00271   34.20526  62.0%  81.6  266s
*35246 23739             175      89.5512137   34.20526  61.8%  82.3  269s
*35251 23730             176      89.5144918   34.20526  61.8%  82.3  269s
 35304 23727   64.51843  170   27   89.51449   34.20526  61.8%  82.4  273s
 37404 25482   57.79775  101   35   89.51449   34.20526  61.8%  82.1  277s
 39139 26966   63.35201  113   19   89.51449   34.20526  61.8%  82.7  280s
 41808 29083   54.63198   86   27   89.51449   34.20526  61.8%  83.7  288s
*42721 29421             191      87.5980131   34.20526  61.0%  83.8  288s
 43529 30046   83.79049  

     0     0   43.71412    0   12          -   43.71412      -     -    4s
     0     0   43.71412    0    9          -   43.71412      -     -    6s
     0     0   43.71412    0   15          -   43.71412      -     -    6s
     0     0   43.71412    0    9          -   43.71412      -     -    7s
     0     0   43.71412    0   12          -   43.71412      -     -    7s
     0     0   43.71412    0    9          -   43.71412      -     -    8s
     0     0   43.71412    0    9          -   43.71412      -     -    9s
     0     2   43.71412    0    9          -   43.71412      -     -   11s
   841   808   43.71412   51   20          -   43.71412      -   125   15s
* 5127  4802             268      85.8656098   43.71412  49.1%  64.5   19s
* 5128  4802             268      83.8965275   43.71412  47.9%  64.5   19s
  5141  4818   43.71412  220   23   83.89653   43.71412  47.9%  64.6   20s
  5303  4926   48.40977  119    9   83.89653   43.71412  47.9%  65.8   25s
  5310  4931   69.52821  

H  409   301                      71.9364086   45.99746  36.1%   232    8s
H  415   206                      64.0651910   45.99746  28.2%   231    8s
H  418   199                      62.9382115   45.99746  26.9%   230    8s
H  438   178                      56.9216164   45.99746  19.2%   225    8s
H 1020   563                      56.8676431   45.99746  19.1%   195    8s
H 1223   708                      56.0792224   45.99746  18.0%   181    9s
H 1448   864                      56.0224415   45.99746  17.9%   173    9s
* 1482   887              50      56.0106775   45.99746  17.9%   170    9s
* 1545   818              64      55.9110317   45.99746  17.7%   166    9s
* 2208  1276              57      55.0084409   45.99746  16.4%   156    9s
  2490  1447   45.99746   70   18   55.00844   45.99746  16.4%   149   12s
H 2774  1654                      55.0043066   45.99746  16.4%   144   12s
H 2792  1009                      52.8814047   45.99746  13.0%   144   12s
  3064  1047   52.86392  

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 23960 rows, 68128 columns and 50300 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [4e+01, 2e+02]
  RHS range        [7e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 20057 rows and 66747 columns
Presolve time: 0.18s
Presolved: 3903 rows, 1921 columns, 9698 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 5.494e+04
 Factor NZ  : 2.283e+05 (roughly 4 MBytes of memory)
 Factor Ops : 1.678e+07 (less than 1 second per iteration)
 Threads    : 22

Barrier performed 0 iterations in 0.23 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 682 iterations and 0.27 seconds
Optimal objective  4.718206012e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_1

...

Presolve time: 0.45s
Presolved: 63204 rows, 15657 columns, 137276 nonzeros

Ordering time: 0.07s

Barrier statistics:
 AA' NZ     : 1.348e+06
 Factor NZ  : 7.008e+06 (roughly 90 MBytes of memory)
 Factor Ops : 9.660e+08 (less than 1 second per iteration)
 Threads    : 22

Barrier performed 0 iterations in 0.79 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 5468 iterations and 0.80 seconds
Optimal objective  5.826784703e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_47_1
  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  47_1  || 200
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 202700 rows, 68128 columns and 396980 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 5e-03]
  Bounds range     [2e+01, 2e+02]
  RHS range        [7e-01, 1e+0

Optimal objective  5.871969892e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
    MIN makespan for DETERMINISTIC    step=1  :   48_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5129 rows, 5152 columns and 18515 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 1933 rows and 1854 columns
Presolve time: 0.05s
Presolved: 3196 rows, 3298 columns, 11857 nonzeros
Variable types: 2721 continuous, 577 integer (577 binary)

Root relaxation: objective 6.301230e+01, 1197 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   63.01230


Solved with primal simplex
Solved in 123 iterations and 0.16 seconds
Optimal objective  6.435618415e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  48_1 || 50
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 53750 rows, 68128 columns and 108080 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 2e-02]
  Bounds range     [4e+01, 1e+02]
  RHS range        [7e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 53592 rows and 67747 columns
Presolve time: 0.11s
Presolved: 158 rows, 381 columns, 759 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.780e+02
 Factor NZ  : 1.849e+03
 Factor Ops : 2.575e+04 (less than 1 second per iteration)
 Threads    : 1

    

   3   6.41050834e+01  6.38478257e+01  1.14e-01 2.08e-17  4.12e-04     0s
   4   6.41050834e+01  6.40901683e+01  7.86e-03 3.12e-17  2.17e-05     0s
   5   6.41050834e+01  6.41050477e+01  6.68e-05 2.78e-17  6.31e-08     0s
   6   6.41050834e+01  6.41050834e+01  8.99e-11 2.78e-17  1.07e-13     0s

Barrier solved model in 6 iterations and 0.16 seconds
Optimal objective 6.41050834e+01

Crossover log...

      20 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

     178 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 0.0000000e+00      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     201    6.4105083e+01   0.000000e+00   0.000000e+00      0s
     201    6.4105083e+01   0.000000e+00   0.000000e+00      0s

Solved with primal simplex
Solved in 143 iterations


  Push phase complete: Pinf 0.0000000e+00, Dinf 2.3128166e-12      5s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11989    6.3594638e+01   0.000000e+00   0.000000e+00      5s
   11989    6.3594638e+01   0.000000e+00   5.000000e-06      5s

Solved with dual simplex
Solved in 14193 iterations and 6.02 seconds
Optimal objective  6.359463780e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  48_1  || 1000
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 997100 rows, 68128 columns and 1937780 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  Bounds range     [4e+01, 1e+02]
  RHS range        [6e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed

In [23]:
print("FINISHED")

FINISHED
